In [42]:
from geopy.distance import great_circle
import geopy.distance
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import pandas as pd

In [43]:
d = {'lat' : [4.75608,3.40314,3.14851,3.14183,3.1465, 3.06751,4.53808,3.88799,4.17635,4.10375],
    'long':[103.40756, 101.55955,101.72299, 101.52673,101.75578,101.47823,103.4552,103.36542,103.27807,103.28159]}
df = pd.DataFrame(d)

In [44]:
df

,lat,long
0,4.75608,103.40756
1,3.40314,101.55955
2,3.14851,101.72299
3,3.14183,101.52673
4,3.14650,101.75578
5,3.06751,101.47823
6,4.53808,103.45520
7,3.88799,103.36542
8,4.17635,103.27807
9,4.10375,103.28159


In [41]:
# Distance callback
def create_distance_callback(dist_matrix):
    # Create a callback to calculate distances between cities.
    def distance_callback(from_node, to_node):
        return int(dist_matrix[from_node][to_node])
    return distance_callback

def main():
    # Point
    lat=df['lat'].values.tolist()
    long=df['long'].values.tolist()
    point_names = list(range(0,len(lat)))
    
    # Distance matrix
    dist_matrix=[]
    for i in range(0,len(lat)):
        dist_list=[]
        for j in range(0, len(lat)):
            coords_1 =(lat[i], long[i])
            coords_2 =(lat[j], long[j])
            dist=float(geopy.distance.vincenty(coords_1, coords_2).km)
            dist_list.append(dist)
        dist_matrix.append(dist_list)
        
    tsp_size = len(point_names)
    num_routes = 1
    depot = 0

    # Create routing model
    if tsp_size > 0:
        routing = pywrapcp.RoutingModel(tsp_size, num_routes, depot)
        search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
        # Create the distance callback.
        dist_callback = create_distance_callback(dist_matrix)
        routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
        # Solve the problem.
        assignment = routing.SolveWithParameters(search_parameters)
        if assignment:
            # Solution distance.
            print ("Total distance: " + str(assignment.ObjectiveValue()) + " km\n")
            # Display the solution.
            # Only one route here; otherwise iterate from 0 to routing.vehicles() - 1
            
            route_number = 0
            node = routing.Start(route_number) # Index of the variable for the starting node.
            route = ''
            while not routing.IsEnd(node):
                # Convert variable indices to node indices in the displayed route.
                route += str(node) + ' -> '
                node = assignment.Value(routing.NextVar(node))
            route += '0'
                #route += str(point_names[routing.IndexToNode(index)]) + ' -> '
                #index = point_names.Value(routing.NextVar(index))
                #route += str(point_names[routing.IndexToNode(index)])
            print ("Route:\n\n" + route)
        else:
            print ('No solution found.')
    else:
        print ('Specify an instance greater than 0.')

if __name__ == '__main__':
    main()

Total distance: 619 km

Route:

0 -> 6 -> 8 -> 9 -> 7 -> 4 -> 2 -> 5 -> 3 -> 1 -> 0
